In [61]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [62]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [63]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [64]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [65]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [66]:
# Create a DataFrame with the scaled data, then set the columns from the original DataFrame and the index. 

scaled_df = pd.DataFrame(scaled_data, 
                         columns=df_market_data.columns, 
                         index=df_market_data.index)

# Copy the crypto names from the original DataFrame Done in previous step.
# Set the coin_id column as index. Done in previous step.


# Display the scaled DataFrame
print("\nScaled DataFrame:")
print(scaled_df.head())


Scaled DataFrame:
              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash             

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [67]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

print("\nList of k values:")
print(k_values)


List of k values:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [68]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
for k in k_values:
    try:
        kmeans = KMeans(n_clusters=k, random_state=0, n_init=10) 
        # 2. Fit the model to the data using `df_market_data_scaled`
        kmeans.fit(scaled_df)
        # 3. Append the model.inertia_ to the inertia list
        inertia_values.append(kmeans.inertia_)
    except Exception as e:
        print(f"Error encountered for k={k}: {e}") #print any errors that occured during execution
    

print("\nInertia values for each k:")
print(inertia_values)
# **CRITICAL: Check lengths!**
print(f"\nLength of k_values: {len(k_values)}")
print(f"Length of inertia_values: {len(inertia_values)}")


Inertia values for each k:
[287.0, 198.5718180979813, 123.19048183836958, 79.02243535120975, 65.30237914162498, 52.10180599356482, 46.03525005555019, 37.284806594046614, 32.16141710107241, 28.075252677898867, 23.89004435537795]

Length of k_values: 11
Length of inertia_values: 11


In [69]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k': k_values, 'inertia': inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

print("\nElbow Data DataFrame:")
print(elbow_df.head())


Elbow Data DataFrame:
   k     inertia
0  1  287.000000
1  2  198.571818
2  3  123.190482
3  4   79.022435
4  5   65.302379


In [70]:
# Plot a line chart with all the inertia values computed with
elbow_curve_plot = elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_values,
    yticks=list(range(0, 250, 25))
)
# the different values of k to visually identify the optimal value for k.
elbow_curve_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [71]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters= 4, random_state=0, n_init=10)

In [72]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(scaled_df)

KMeans(n_clusters=4, n_init=10, random_state=0)

In [73]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
cluster_predictions = kmeans_model.predict(scaled_df)

# Print the resulting array of cluster values.
print("\nCluster Predictions:")
print(cluster_predictions)


Cluster Predictions:
[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [74]:
# Create a copy of the scaled DataFrame
clustered_df = scaled_df.copy()

In [75]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
clustered_df['cluster'] = cluster_predictions

# Display the copy of the scaled DataFrame
print("\nClustered DataFrame:")
print(clustered_df.head())


Clustered DataFrame:
              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash          

In [76]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scatter_plot = clustered_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters"
)
scatter_plot  # Display the plot in the notebook


:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [77]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3, random_state=0)

In [78]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(scaled_df)

# View the scaled PCA data
print("\nScaled PCA Data:")
print(pca_data[:5])  # Display first 5 rows


Scaled PCA Data:
[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]]


In [79]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance_ratio = pca.explained_variance_ratio_
print("\nExplained Variance Ratio:")
print(explained_variance_ratio)


Explained Variance Ratio:
[0.3719856  0.34700813 0.17603793]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 
3719856 + 0.34700813 + 0.17603793 = 0.89503166

This means that the three principal components capture about 89.5% of the total variance in the original scaled data. By reducing the dimensionality to three components, you've retained a large portion of the information while simplifying the data. The remaining ~10.5% of the variance has been discarded.

In [80]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'], index=scaled_df.index)

# Copy the crypto names from the original scaled DataFrame (already done by setting index)
# Set the coin_id column as index (already done when creating the DataFrame)

# Display the scaled PCA DataFrame
print("\nScaled PCA DataFrame:")
print(pca_df.head())



Scaled PCA DataFrame:
                   PC1       PC2       PC3
coin_id                                   
bitcoin      -0.600667  0.842760  0.461595
ethereum     -0.458261  0.458466  0.952877
tether       -0.433070 -0.168126 -0.641752
ripple       -0.471835 -0.222660 -0.479053
bitcoin-cash -1.157800  2.041209  1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [81]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = list(range(1, 12))

In [82]:
# Create an empty list to store the inertia values
inertia_values_pca = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values_pca:
    try:
        kmeans_pca = KMeans(n_clusters=k, random_state=0, n_init=10)
        kmeans_pca.fit(pca_df)
        inertia_values_pca.append(kmeans_pca.inertia_)
    except Exception as e:
        print(f"Error encountered for k={k}: {e}")

In [83]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {'k': k_values_pca, 'inertia': inertia_values_pca}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df_pca = pd.DataFrame(elbow_data_pca)

In [84]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_curve_plot_pca = elbow_df_pca.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve (PCA Data)",
    xticks=k_values_pca,
    yticks=list(range(0, int(max(inertia_values_pca)) + 25, 25))
)
elbow_curve_plot_pca  # Display the plot in the notebook

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 
 Based on the elbow curve generated using the PCA-transformed data, the best value for k appears to be 4. The "elbow" in the curve, where the rate of decrease in inertia starts to diminish, is most prominent at k=4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 
  The best k value remains at 4 when using the PCA-transformed data, which is the same value we identified using the original scaled data. However, it's important to note that the inertia values are generally lower in the PCA-transformed data's elbow curve compared to the original scaled data. This suggests that the clusters formed using the PCA data are more tightly packed and well-defined, likely due to the dimensionality reduction focusing on the most important variance components. Even though K is the same, the clusters are probably shaped differently in the original dataset and the PCA dataset.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [85]:
# Initialize the K-Means model using the best value for k
kmeans_model_pca = KMeans(n_clusters=4, random_state=0, n_init=10) #best_k determined to be 4

In [86]:
# Fit the K-Means model using the PCA data
kmeans_model_pca.fit(pca_df)

KMeans(n_clusters=4, n_init=10, random_state=0)

In [87]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
cluster_predictions_pca = kmeans_model_pca.predict(pca_df)

# Print the resulting array of cluster values.
print("\nCluster Predictions (PCA):")
print(cluster_predictions_pca)


Cluster Predictions (PCA):
[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [90]:
# Create a copy of the scaled PCA DataFrame
clustered_pca_df = pca_df.copy()
# Add a new column to the copy of the PCA DataFrame with the predicted clusters

clustered_pca_df['cluster'] = cluster_predictions_pca




# Display the copy of the scaled PCA DataFrame
print("\nClustered PCA DataFrame:")
print(clustered_pca_df.head())


Clustered PCA DataFrame:
                   PC1       PC2       PC3  cluster
coin_id                                            
bitcoin      -0.600667  0.842760  0.461595        2
ethereum     -0.458261  0.458466  0.952877        2
tether       -0.433070 -0.168126 -0.641752        0
ripple       -0.471835 -0.222660 -0.479053        0
bitcoin-cash -1.157800  2.041209  1.859715        2


### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [91]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scatter_plot_pca = clustered_pca_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (PCA Data)"
)
scatter_plot_pca # Display the plot in the notebook

# Display the copy of the scaled PCA DataFrame


:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [92]:
# Composite plot to contrast the Elbow curves
composite_elbow_plot = elbow_curve_plot + elbow_curve_plot_pca
composite_elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [93]:
# Composite plot to contrast the clusters
composite_cluster_plot = scatter_plot + scatter_plot_pca
composite_cluster_plot

:Layout
   .NdOverlay.I  :NdOverlay   [cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features (as achieved through PCA) in K-Means clustering simplifies the data representation, leading to the formation of more distinct and potentially more meaningful clusters. In our case, the PCA plot shows clusters that are more clearly separated than in the original scatter plot. However, this simplification comes at a cost. Because PCA reduces the dimensions of the feature space, the algorithm has less information about the points that are being clustered. If important dimensions are dropped, then the generated clusters may be less optimal than clusters generated using a larger feature set.